In [ ]:
"""
    -- Description:
        
        (1) This notebook shows a trend following based futures trading strategy.
        
        (2) A bullish trend is defined by the fast moving average of close price surpassing the slow moving 
        
        average. A bearish trend is defind by the fast moving average falling below the slow moving average.
        
        (3) The strategy will enter either long or short positions once the price hits a N-day new high/low.
        
        (4) Exit of positions occur once the trend is over, or the price goes against the position significantly.
        
        We quantify this by 3 * std.
        
        (5) The position size will be determined by the std. of historical price change, multipled by point value.
        
        (6) Both commmission and slippage will be considered during simulation.
        
        
    -- Prerequisite:
        
        Quantopian IDE
        
    -- Auther:
        
        Fang Fan (Roy)
        
    -- Date:
        
        August 28, 2020
    
"""

In [1]:
import zipline
from zipline.api import future_symbol, set_benchmark, set_commission, set_slippage
from zipline.api import schedule_function, date_rules, time_rules, continuous_future, order_target
from datetime import datetime
import pytz
import pandas as pd
import numpy as np
from zipline.finance.commission import PerTrade, PerContract
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage, VolatilityVolumeShare

In [2]:
# Model Settings:
start_portfolio = 50000000
risk_factor = 0.02
stop_distance = 3
breakout_window = 50
vola_window = 40
slow_ma = 80
fast_ma = 40
enable_commission = True
enable_slippage = True

In [3]:
def position_size(portfolio_value, std, point_value):
    """
    Input: Current marked-to-market portfolio value, futures std of price changes, and contract multiplier
    Output: # of contracts we should enter
    """
    target_variation = portfolio_value * risk_factor
    contract_variation = std * point_value
    if contract_variation == 0:
        return 0
    
    contracts = target_variation / contract_variation
    return int(np.nan_to_num(contracts))

In [4]:
def roll_futures(context, data):
    """
    This function performs futures position rolling when necesssary.
    Note that if the positions are set to change by core logic, it won't roll positions.
    """
    open_orders = zipline.api.get_open_orders()
    
    for held_contract in context.portfolio.positions:
        # Don't close if not near the auto close date, or if set to change by core logic.
        if held_contract in open_orders:
            continue
        
        days_to_auto_close = (held_contract.auto_close_date.date() - data.current_session.date()).days
        if days_to_auto_close > 5:
            continue
            
        # Position rolling
        continuation = continuous_future(held_contract.root_symbol, offset=0, roll="volume", adjustment="mul")
        continuation_contract = data.current(continuation, "contract")
        
        if continuation_contract != held_contract:
            pos_size = context.portfolio.positions[held_contract].amount
            order_target(held_contract, 0)
            order_target(continuation_contract, pos_size)
            

In [9]:
def initialize(context):
    # Cost Settings
    if enable_commission:
        commission_model = PerContract(cost=0.85, exchange_fee=1.5)
    else:
        commission_model = PerTrade(0.0)
    
    set_commission(us_futures=commission_model)
    
    if enable_slippage:
        slippage_model = VolatilityVolumeShare(volume_limit=0.2)
    else:
        slippage_model = FixedSlippage(spread=0.0)
    
    set_slippage(us_futures=slippage_model)
    
    # Markets to Trade in
    # We ensure diversity by selecting futures across different sectors
    
    markets = ["CL", "BO", "TN", "TS", "FF", "FI", "AD", "BD", "CN", "CM", "DJ", "YM", "EU", "ET", "LH", "LC", "LB","MG", "EI", "MB", "ME", "NQ", "ND", "TB", "US", "TY", "TU", "UB", "HU", "VX", "WC", "MW", "YS", "AI","BP","CD","HG","QM","EC","EE","ED","EL","FC","GC","XG","JY","JE"]
    
    # Make a list of all continuations
    context.universe = [continuous_future(market, offset=0, roll="volume", adjustment="mul") for market in markets]
    
    # For each market, keep track of the best position reading
    context.highest_in_position = {market:0 for market in markets}
    context.lowest_in_position = {market:0 for market in markets}
    
    # Schedule the daily trading:
    schedule_function(daily_trade, date_rules.every_day(), time_rules.market_close())

In [11]:
def daily_trade(context, data):
    # Record open positions if any (symbol: futures)
    open_pos = {pos.root_symbol: pos for pos in context.portfolio.positions}
    
    for continuation in context.universe:
        root = continuation.root_symbol
        
        h = data.history(continuation, fields=["close", "volume"], frequency="1d", bar_count=250)
        h["trend"] = h["close"].ewm(span=fast_ma).mean() > h["close"].ewm(span=slow_ma).mean()
        std = h["close"].diff()[:-vola_window].std()
        
        # Close open positions when conditions are met
        if root in open_pos:
            p = context.portfolio.positions[open_pos[root]]
            if p.amount > 0:
                # For Long Positions
                if context.highest_in_position[root] == 0: # First day holding the position
                    context.highest_in_position[root] = p.cost_basis
                else:
                    context.highest_in_position[root] = max(h["close"].iloc[-1], context.highest_in_position[root])
                stop = context.highest_in_position[root] - std * stop_distance
                if h["close"].iloc[-1] < stop or h["trend"].iloc[-1] == False:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.highest_in_position[root] = 0
            else: 
                # For Short Positions
                if context.lowest_in_position[root] == 0:
                    context.lowest_in_position[root] = p.cost_basis
                else:
                    context.lowest_in_position[root] = min(context.lowest_in_position[root], h["close"].iloc[-1])
                stop = context.lowest_in_position[root] + std * stop_distance
                if h["close"].iloc[-1] > stop or h["trend"].iloc[-1] == True:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.lowest_in_position[root] = 0
        else:
            # Open positions when conditions are met
            if h["trend"].iloc[-1] == True: 
                # Bullish Trend, hit new high then long
                if h["close"].iloc[-1] == h["close"][-breakout_window:].max():
                    contract = data.current(continuation, "contract")
                    contracts_to_trade = position_size(context.portfolio.portfolio_value, std, contract.price_multiplier)
                    # Practical limit to trade: maximum 20% of avg. daily volume
                    contracts_cap = int(h["volume"][-20:].mean() * 0.2)
                    order_target(contract, min(contracts_to_trade, contracts_cap))
            else: 
                # Bearish Trend, hit new low then short
                if h["close"].iloc[-1] == h["close"][-breakout_window:].min():
                    contract = data.current(continuation, "contract")
                    contracts_to_trade = position_size(context.portfolio.portfolio_value, std, contract.price_multiplier)
                    contracts_cap = int(h["volume"][-20:].mean() * 0.2)
                    order_target(contract, -min(contracts_to_trade, contracts_cap))
                    
    # If we have open positions, check for rolls:
    if len(open_pos) > 0:
        roll_futures(context, data)